In [1]:
!pip install pyspark matplotlib pandas numpy scikit-learn
# Import required libraries
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, CountVectorizer, IDF, ChiSqSelector, Normalizer, StringIndexer
from pyspark.ml.classification import LinearSVC
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random

# Set a fixed random seed for reproducibility
seed = 42
random.seed(seed)
np.random.seed(seed)



In [3]:
# Initialize SparkSession
spark = SparkSession \
    .builder \
    .appName("DIC EX 2 - group 36") \
    .getOrCreate()

print("Spark session initialized")



Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: java.lang.UnsupportedOperationException: getSubject is supported only if a security manager is allowed
	at java.base/javax.security.auth.Subject.getSubject(Subject.java:347)
	at org.apache.hadoop.security.UserGroupInformation.getCurrentUser(UserGroupInformation.java:577)
	at org.apache.spark.util.Utils$.$anonfun$getCurrentUserName$1(Utils.scala:2416)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.util.Utils$.getCurrentUserName(Utils.scala:2416)
	at org.apache.spark.SparkContext.<init>(SparkContext.scala:329)
	at org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
	at java.base/jdk.internal.reflect.DirectConstructorHandleAccessor.newInstance(DirectConstructorHandleAccessor.java:62)
	at java.base/java.lang.reflect.Constructor.newInstanceWithCaller(Constructor.java:501)
	at java.base/java.lang.reflect.Constructor.newInstance(Constructor.ja

In [ ]:
# Load the development dataset
print("Loading review data...")
data_path = "hdfs:///user/dic25_shared/amazon-reviews/full/reviews_devset.json"
df = spark.read.json(data_path)

# Display schema and sample data
print("Dataset Schema:")
df.printSchema()
print("\nSample data:")
df.select("reviewText", "category").show(5, truncate=True)

# Function to check dataset statistics
def print_stats(df):
    total_reviews = df.count()
    category_counts = df.groupBy("category").count().orderBy("count", ascending=False)
    
    print(f"Total number of reviews: {total_reviews}")
    print("Category distribution:")
    category_counts.show(20, truncate=False)
    
    return category_counts

# Get dataset statistics
category_counts = print_stats(df)

# Convert category_counts to Pandas for visualization
category_dist = category_counts.toPandas()
top_10_categories = category_dist.head(10)

# Visualization of category distribution
plt.figure(figsize=(12, 6))
plt.bar(top_10_categories['category'], top_10_categories['count'])
plt.xticks(rotation=90)
plt.title('Distribution of Top 10 Review Categories')
plt.xlabel('Category')
plt.ylabel('Count')
plt.tight_layout()
plt.savefig('category_distribution.png')
plt.close()

# Display the plot
from IPython.display import Image
Image(filename='category_distribution.png')



In [ ]:
# Data Preparation: Split into training, validation, and test sets
print("Splitting data into training, validation, and test sets...")
train_data, temp_data = df.randomSplit([0.7, 0.3], seed=seed)
validation_data, test_data = temp_data.randomSplit([0.5, 0.5], seed=seed)

print(f"Training set size: {train_data.count()}")
print(f"Validation set size: {validation_data.count()}")
print(f"Test set size: {test_data.count()}")

# Load the top features selected in Part 2
print("Loading selected features from output_ds.txt...")
with open("output_ds.txt", "r") as f:
    selected_features = f.read().strip().split()

print(f"Loaded {len(selected_features)} features from output_ds.txt")
print(f"Sample features: {selected_features[:10]}")

print("Building the ML Pipeline for classification...")

# Create a pipeline similar to Part 2 but configured for classification
# 1. Convert category to numeric labels
label_indexer = StringIndexer(inputCol="category", outputCol="label")

# 2. Text preprocessing (similar to Part 2)
tokenizer = RegexTokenizer(inputCol="reviewText", outputCol="tokens", 
                          pattern="[\\s\\t\\d\\(\\)\\[\\]\\{\\}\\.\\!\\?\\,\\;\\:\\+\\=\\-\\_\\\"\\'`\\~\\#\\@\\&\\*\\%\\€\\$\\§\\\\\\/]+")

# 3. Load stopwords
def load_stopwords(path: str) -> list[str]:
    stopwords = set()
    with open(path, "r", encoding="utf-8") as f:
        stopwords = set(line.strip() for line in f if line.strip())
    return list(stopwords)

# 4. Remove stopwords
stopwords_remover = StopWordsRemover(inputCol="tokens", outputCol="tokens_filtered", 
                                    stopWords=load_stopwords("stopwords.txt"))

# 5. Create term frequency vectors
count_vectorizer = CountVectorizer(inputCol="tokens_filtered", outputCol="tf")

# 6. Calculate IDF
idf = IDF(inputCol="tf", outputCol="tf_idf")

# 7. Feature selection using Chi-Square
# Here we'll use two different feature dimensionalities for comparison
# 7.1 - 2000 features (as required in the assignment)
chi_sq_selector_2000 = ChiSqSelector(numTopFeatures=2000, 
                                    featuresCol="tf_idf", 
                                    outputCol="selected_features",
                                    labelCol="label")

# 7.2 - A much smaller dimensionality for comparison (500 features)
chi_sq_selector_500 = ChiSqSelector(numTopFeatures=500, 
                                   featuresCol="tf_idf", 
                                   outputCol="selected_features",
                                   labelCol="label")

# 8. Vector normalization (required by the assignment)
normalizer = Normalizer(inputCol="selected_features", outputCol="normalized_features", p=2.0)

# 9. Define the SVM classifier
svm = LinearSVC(featuresCol="normalized_features", labelCol="label", predictionCol="prediction")

# Build the pipeline with 2000 features
pipeline_2000 = Pipeline(stages=[
    label_indexer,
    tokenizer,
    stopwords_remover,
    count_vectorizer,
    idf,
    chi_sq_selector_2000,
    normalizer,
    svm
])

# Build the pipeline with 500 features
pipeline_500 = Pipeline(stages=[
    label_indexer,
    tokenizer,
    stopwords_remover,
    count_vectorizer,
    idf,
    chi_sq_selector_500,
    normalizer,
    svm
])



In [ ]:
# Set up the parameter grid for the SVM
print("Setting up parameter grid for SVM optimization...")
param_grid = ParamGridBuilder() \
    .addGrid(svm.regParam, [0.01, 0.1, 1.0]) \
    .addGrid(svm.standardization, [True, False]) \
    .addGrid(svm.maxIter, [10, 50]) \
    .build()

# Create an evaluator for model assessment
evaluator = MulticlassClassificationEvaluator(
    labelCol="label", 
    predictionCol="prediction", 
    metricName="f1"
)

# Create cross-validators for both feature dimensionalities
cv_2000 = CrossValidator(
    estimator=pipeline_2000,
    estimatorParamMaps=param_grid,
    evaluator=evaluator,
    numFolds=3,
    seed=seed
)

cv_500 = CrossValidator(
    estimator=pipeline_500,
    estimatorParamMaps=param_grid,
    evaluator=evaluator,
    numFolds=3,
    seed=seed
)

# Train and evaluate models
results = []

# Function to extract parameter settings from a model
def extract_params(pipeline_model):
    svm_model = None
    for stage in pipeline_model.stages:
        if isinstance(stage, type(LinearSVC())):
            svm_model = stage
            break
    
    if svm_model:
        return {
            "regParam": svm_model.getRegParam(),
            "standardization": svm_model.getStandardization(),
            "maxIter": svm_model.getMaxIter()
        }
    return None

# Train model with 2000 features
print("\nTraining model with 2000 features...")
print("This may take some time...")
cv_model_2000 = cv_2000.fit(train_data)

# Get the best model
best_model_2000 = cv_model_2000.bestModel
params_2000 = extract_params(best_model_2000)

# Apply the best model to the test set
predictions_2000 = best_model_2000.transform(test_data)

# Evaluate the model
f1_score_2000 = evaluator.evaluate(predictions_2000)

# Store results
results.append({
    "feature_set": "2000 features",
    "regParam": params_2000["regParam"],
    "standardization": params_2000["standardization"],
    "maxIter": params_2000["maxIter"],
    "f1_score": f1_score_2000
})

print(f"\nBest parameters for 2000 features:")
print(f"  regParam: {params_2000['regParam']}")
print(f"  standardization: {params_2000['standardization']}")
print(f"  maxIter: {params_2000['maxIter']}")
print(f"  F1 score on test set: {f1_score_2000:.4f}")



In [ ]:
# Train model with 500 features
print("\nTraining model with 500 features...")
print("This may take some time...")
cv_model_500 = cv_500.fit(train_data)

# Get the best model
best_model_500 = cv_model_500.bestModel
params_500 = extract_params(best_model_500)

# Apply the best model to the test set
predictions_500 = best_model_500.transform(test_data)

# Evaluate the model
f1_score_500 = evaluator.evaluate(predictions_500)

# Store results
results.append({
    "feature_set": "500 features",
    "regParam": params_500["regParam"],
    "standardization": params_500["standardization"],
    "maxIter": params_500["maxIter"],
    "f1_score": f1_score_500
})

print(f"\nBest parameters for 500 features:")
print(f"  regParam: {params_500['regParam']}")
print(f"  standardization: {params_500['standardization']}")
print(f"  maxIter: {params_500['maxIter']}")
print(f"  F1 score on test set: {f1_score_500:.4f}")

# Save confusion matrix for the best model
# Find which model performed better
best_predictions = predictions_2000 if f1_score_2000 > f1_score_500 else predictions_500
best_feature_set = "2000 features" if f1_score_2000 > f1_score_500 else "500 features"



In [ ]:
# Get label mapping from the indexer
label_mapping = {idx: cat for idx, cat in enumerate(label_indexer.fit(df).labels)}

# Convert predictions to a pandas DataFrame for easier analysis
pred_df = best_predictions.select("category", "prediction", "label").toPandas()
true_labels = pred_df["label"]
pred_labels = pred_df["prediction"]

# Calculate confusion matrix for top categories
top_categories = category_counts.limit(10).toPandas()['category'].tolist()
category_to_idx = {cat: idx for idx, cat in enumerate(label_indexer.fit(df).labels)}
top_indices = [category_to_idx[cat] for cat in top_categories]

# Initialize confusion matrix
conf_matrix = np.zeros((len(top_indices), len(top_indices)), dtype=int)

# Fill confusion matrix
for i, true_cat in enumerate(top_indices):
    for j, pred_cat in enumerate(top_indices):
        conf_matrix[i, j] = sum((true_labels == true_cat) & (pred_labels == pred_cat))

# Visualize confusion matrix
plt.figure(figsize=(12, 10))
plt.imshow(conf_matrix, interpolation='nearest', cmap=plt.cm.Blues)
plt.title(f'Confusion Matrix - Top 10 Categories ({best_feature_set})')
plt.colorbar()
tick_marks = np.arange(len(top_categories))
plt.xticks(tick_marks, top_categories, rotation=90)
plt.yticks(tick_marks, top_categories)
plt.ylabel('True Category')
plt.xlabel('Predicted Category')
plt.tight_layout()
plt.savefig(f'confusion_matrix_{best_feature_set.replace(" ", "_")}.png')
plt.close()

# Display the plot
Image(filename=f'confusion_matrix_{best_feature_set.replace(" ", "_")}.png')



In [ ]:
# Summarize results in a table
print("\nSummary of Model Performance:")
results_df = pd.DataFrame(results)
print(results_df)

# Visualization of results
plt.figure(figsize=(10, 6))
bars = plt.bar(results_df['feature_set'], results_df['f1_score'])

# Add value labels on top of bars
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height + 0.01,
             f'{height:.4f}', ha='center', va='bottom')

plt.title('F1 Score Comparison: 2000 vs 500 Features')
plt.ylabel('F1 Score')
plt.xlabel('Feature Set')
plt.ylim(0, max(results_df['f1_score']) * 1.1)  # Add 10% padding at the top
plt.savefig('f1_comparison.png')
plt.close()

# Display the plot
Image(filename='f1_comparison.png')

# Analyze the effect of different parameters
# Extract metrics from cross-validation results for 2000 features
cv_metrics_2000 = []
for i, metric in enumerate(cv_model_2000.avgMetrics):
    params = param_grid[i]
    reg_param = params[svm.regParam]
    standardization = params[svm.standardization]
    max_iter = params[svm.maxIter]
    
    cv_metrics_2000.append({
        'regParam': reg_param,
        'standardization': standardization,
        'maxIter': max_iter,
        'f1_score': metric,
        'feature_set': '2000 features'
    })

# Extract metrics from cross-validation results for 500 features
cv_metrics_500 = []
for i, metric in enumerate(cv_model_500.avgMetrics):
    params = param_grid[i]
    reg_param = params[svm.regParam]
    standardization = params[svm.standardization]
    max_iter = params[svm.maxIter]
    
    cv_metrics_500.append({
        'regParam': reg_param,
        'standardization': standardization,
        'maxIter': max_iter,
        'f1_score': metric,
        'feature_set': '500 features'
    })

# Combine metrics
all_metrics = cv_metrics_2000 + cv_metrics_500
metrics_df = pd.DataFrame(all_metrics)

# Analyze effect of regularization parameter
plt.figure(figsize=(12, 6))
for feature_set in ['2000 features', '500 features']:
    subset = metrics_df[metrics_df['feature_set'] == feature_set]
    reg_effect = subset.groupby('regParam')['f1_score'].mean()
    plt.plot(reg_effect.index, reg_effect.values, marker='o', label=feature_set)

plt.title('Effect of Regularization Parameter on F1 Score')
plt.xlabel('Regularization Parameter (regParam)')
plt.ylabel('Average F1 Score')
plt.xscale('log')
plt.legend()
plt.grid(True)
plt.savefig('regparam_effect.png')
plt.close()

# Display the plot
Image(filename='regparam_effect.png')

# Analyze effect of standardization
plt.figure(figsize=(10, 6))
std_effect = metrics_df.groupby(['feature_set', 'standardization'])['f1_score'].mean().reset_index()

# Plot for 2000 features
std_effect_2000 = std_effect[std_effect['feature_set'] == '2000 features']
plt.bar([0, 2], std_effect_2000['f1_score'], width=0.4, label='2000 features')

# Plot for 500 features
std_effect_500 = std_effect[std_effect['feature_set'] == '500 features']
plt.bar([0.5, 2.5], std_effect_500['f1_score'], width=0.4, label='500 features')

plt.title('Effect of Standardization on F1 Score')
plt.xlabel('Standardization')
plt.xticks([0.25, 2.25], ['False', 'True'])
plt.ylabel('Average F1 Score')
plt.legend()
plt.grid(axis='y')
plt.savefig('standardization_effect.png')
plt.close()

# Display the plot
Image(filename='standardization_effect.png')

# Analyze effect of maximum iterations
plt.figure(figsize=(10, 6))
iter_effect = metrics_df.groupby(['feature_set', 'maxIter'])['f1_score'].mean().reset_index()

# Plot for 2000 features
iter_effect_2000 = iter_effect[iter_effect['feature_set'] == '2000 features']
plt.bar([0, 2], iter_effect_2000['f1_score'], width=0.4, label='2000 features')

# Plot for 500 features
iter_effect_500 = iter_effect[iter_effect['feature_set'] == '500 features']
plt.bar([0.5, 2.5], iter_effect_500['f1_score'], width=0.4, label='500 features')

plt.title('Effect of Maximum Iterations on F1 Score')
plt.xlabel('Maximum Iterations')
plt.xticks([0.25, 2.25], ['10', '50'])
plt.ylabel('Average F1 Score')
plt.legend()
plt.grid(axis='y')
plt.savefig('maxiter_effect.png')
plt.close()

# Display the plot
Image(filename='maxiter_effect.png')

# Determine the overall best model configuration
best_config = max(results, key=lambda x: x['f1_score'])
print("\nBest Overall Configuration:")
print(f"Feature Set: {best_config['feature_set']}")
print(f"regParam: {best_config['regParam']}")
print(f"standardization: {best_config['standardization']}")
print(f"maxIter: {best_config['maxIter']}")
print(f"F1 Score: {best_config['f1_score']:.4f}")

# Save the best model parameters
with open("best_model_params.txt", "w") as f:
    f.write(f"Feature Set: {best_config['feature_set']}\n")
    f.write(f"regParam: {best_config['regParam']}\n")
    f.write(f"standardization: {best_config['standardization']}\n")
    f.write(f"maxIter: {best_config['maxIter']}\n")
    f.write(f"F1 Score: {best_config['f1_score']:.4f}\n")

print("\nBest model parameters saved to 'best_model_params.txt'")

# Conclusion
print("\nConclusion:")
print("We have successfully implemented a text classification pipeline using Spark ML.")
print("The pipeline includes text preprocessing, feature extraction with TF-IDF, and SVM classification.")
print("We compared two feature dimensions (2000 vs 500 features) and varied SVM parameters.")
print(f"The best model achieved an F1 score of {best_config['f1_score']:.4f} on the test set.")

# Stop Spark session
spark.stop()
print("Spark session stopped")